In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup
import json

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
response = requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param=42101&bdate=20080101&edate=20080131&state=42').text

In [2]:
# 2008~2020 Pennsylvania pollutant data extraction
# pollutant를 바꿔가면서 추출
    # 42101 CO - done
    # 42401 SO2 - 2012.10까지밖에 추출 안됨, 일단 밑에것부터 하자
    # 42602 NO2 - done
    # 44201 O3 - done
    # 81102 PM10 - done
    # 88101 PM2.5 - done

import json

pollutant = 42401
pollutant_name = 'Sulfur dioxide'
months_30 = [4, 6, 9, 11]
months_31 = [1, 3, 5, 7, 8, 10, 12]

for year in range(2013, 2021):   # 여기 바꾸기
    for month in range(2, 13): 
        if month in months_30: 
            response = requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param={}&bdate={}{:02d}01&edate={}{:02d}30&state=42'.format(pollutant, year, month, year, month), verify=False)
        elif month in months_31: 
            response = requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param={}&bdate={}{:02d}01&edate={}{:02d}31&state=42'.format(pollutant, year, month, year, month), verify=False)
        else:   # 2월인 경우
            if year % 4 == 0:   # 윤년인 경우 따로 분리 - 29일까지 있음
                response = requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param={}&bdate={}{:02d}01&edate={}{:02d}29&state=42'.format(pollutant, year, month, year, month), verify=False)
            else:   # 28일까지 있는 경우
                response = requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param={}&bdate={}{:02d}01&edate={}{:02d}28&state=42'.format(pollutant, year, month, year, month), verify=False)

        html = response.text
        python_obj = json.loads(html)

        # key들을 list로 변환
        list_key = []
        for keys in python_obj['Data'][0].keys(): 
            list_key.append(keys)

        # create empty df, list_key as columns (월별로 저장하기 위해, 월별로 새로운 empty dataframe 생성)
        df = pd.DataFrame(columns = list_key)

        print("Data append for {}.{:02d} start ------".format(year, month))
        # value들을 list로 변환하면서 final df에 append
        for idx in range(len(python_obj['Data'])): 
            list_value = []
            for values in python_obj['Data'][idx].values(): 
                list_value.append(values)
            list_value_df = pd.DataFrame(list_value, index = list_key)
            list_value_df = list_value_df.transpose()
            
            df = pd.concat([df, list_value_df], axis = 0)
        
        df.reset_index(drop = True, inplace = True)
        print("END. Data length for {}.{:02d} is {}".format(year, month, idx))
        df.to_csv("D:\\SNUlab\\0. data\\Pennsylvania pollutant data (EPA AQS API)\\{}\\{}{:02d}.csv".format(pollutant_name, year, month), index = False) 
        print("{} data for {}.{:02d} saved".format(pollutant_name, year, month))

c:\Users\admin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aqs.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Data append for 2013.02 start ------
END. Data length for 2013.02 is 253332
Sulfur dioxide data for 2013.02 saved


c:\Users\admin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aqs.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Data append for 2013.03 start ------
END. Data length for 2013.03 is 279525
Sulfur dioxide data for 2013.03 saved


c:\Users\admin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aqs.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

Data append for 2014.08 start ------
END. Data length for 2014.08 is 270815
Sulfur dioxide data for 2014.08 saved


c:\Users\admin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aqs.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Data append for 2014.09 start ------
END. Data length for 2014.09 is 262079
Sulfur dioxide data for 2014.09 saved


c:\Users\admin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aqs.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Data append for 2014.10 start ------


In [ ]:
# 2008.01.01 ~ 2008.01.31 CO2 test
import json

response = requests.get('https://aqs.epa.gov/data/api/sampleData/byState?email=zio0323gm@gmail.com&key=indigowren21&param=42101&bdate=20080101&edate=20080131&state=42')
html = response.text
python_obj = json.loads(html)

# key들을 list로 변환
list_key = []
for keys in python_obj['Data'][0].keys(): 
    list_key.append(keys)

# create empty df, list_key as columns
df = pd.DataFrame(columns = list_key)

# value들을 list로 변환하면서 final df에 append
for idx in range(len(python_obj['Data'])): 
    list_value = []
    for values in python_obj['Data'][idx].values(): 
        list_value.append(values)
    list_value_df = pd.DataFrame(list_value, index = list_key)
    list_value_df = list_value_df.transpose()
    
    df = pd.concat([df, list_value_df], axis = 0)

df.reset_index(drop = True, inplace = True)
df

In [ ]:
# https://aqs.epa.gov/data/api/list/parametersByClass?email=test@aqs.api&key=test&pc=CRITERIA
# pollutants parameter code